In [1]:
import cupy as cp
import pandas as pd
import numpy as np
import torch
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score

In [2]:
sortedFeatureCorr = pd.read_csv("dataset/corr.csv")
data = pd.read_csv("dataset/preprocessed.csv")

In [3]:
import time
def stopWatch(value):
    valueD = (((value/365)/24)/60)
    Days = int (valueD)
    valueH = (valueD-Days)*365
    Hours = int(valueH)
    valueM = (valueH - Hours)*24
    Minutes = int(valueM)
    valueS = (valueM - Minutes)*60
    Seconds = int(valueS)
    #start=time.time()
    #end=time.time()
    #stopWatch(end-start)
    print (Days,";",Hours,":",Minutes,";",Seconds)

In [4]:
first_feature_count =           11
last_feature_count =            10

In [5]:
a = sortedFeatureCorr.iloc[:, 0].values

In [6]:
for i in range(first_feature_count , sortedFeatureCorr.shape[0] - last_feature_count):
    data = data.drop(a[i], axis=1)
data=data.drop(list(data.columns)[0], axis=1)


In [7]:
data_np = data.iloc[:-2, :].values
#target = data.iloc[:, -1].values

print("feature " + str(data_np.shape))

#print("target " + str(target.shape))



feature (762104, 21)


In [8]:
def kd_tree(data, counter, arr):
    if(counter == 1):
        arr.append(data)
    if (counter > 0):
        covArr = data[:-1].var(axis=0)
        maxElement = covArr.argmax(axis=0) 
        data = data[data[:,maxElement].argsort()]
        half = int(len (data) / 2)
        print(data [0])
        print(data.shape)
        print("max covarıance feature ındex " + str(maxElement) + " ve half degerı " + str(half))
        kd_tree(data[: half], counter - 1,arr)
        kd_tree(data[half :], counter - 1, arr)
    
kdTreeArr = []
kd_tree(data_np, 3, kdTreeArr)

[ 2  2 70  1  0  0  0  0  0  0  1  1  0  0  1  0  0  1  0  1  2]
(762104, 21)
max covarıance feature ındex 2 ve half degerı 381052
[ 2  2 70  1  0  0  0  0  0  0  1  1  0  0  1  0  0  1  0  1  2]
(381052, 21)
max covarıance feature ındex 2 ve half degerı 190526
[ 2  2 70  1  0  0  0  0  0  0  1  1  0  0  1  0  0  1  0  1  2]
(190526, 21)
max covarıance feature ındex 2 ve half degerı 95263
[  1   1 280   1   0   0   0   0   0   0   1   1   0   1   0   0   0   1
   0   1   4]
(190526, 21)
max covarıance feature ındex 2 ve half degerı 95263
[  2   2 358   1   0   0   0   0   0   0   1   1   0   0   1   0   0   1
   0   1   5]
(381052, 21)
max covarıance feature ındex 2 ve half degerı 190526
[  2   2 358   1   0   0   0   0   0   0   1   1   0   0   1   0   0   1
   0   1   5]
(190526, 21)
max covarıance feature ındex 2 ve half degerı 95263
[  2   5 470   1   0   0   0   0   0   1   0   0   0   0   1   0   0   1
   0   0   5]
(190526, 21)
max covarıance feature ındex 2 ve half degerı 95263

In [9]:
len(kdTreeArr[3])

190526

In [10]:
from torch import nn

class network(nn.Module):
    def __init__(self):
        super().__init__()        
        # Inputs to hidden layer linear transformation
        self.output = nn.Linear(first_feature_count + last_feature_count - 1, 121936) 
    def forward(self, x):
        x = self.output(x)
        return x

In [11]:
knn = network()
knn = knn.cuda()



In [12]:
def normalize_ecnodings(encodings):
    ref_norms = np.linalg.norm(encodings, axis=0)
    return encodings / ref_norms

In [13]:
def select_k(similarityMat, k_neighbour, train, trueLabel):
    truePred = 0
    for houseIndex in range(len(similarityMat)):
        maxIndices = np.argpartition(similarityMat[houseIndex], -1 *k_neighbour)[-1 * k_neighbour:]
        neigVote = train[houseIndex][maxIndices]
        pred = np.bincount(neigVote).argmax()
        if(trueLabel[houseIndex] == pred):
            truePred += 1
        
    return truePred / len(similarityMat)

In [14]:
def similarity(X_test):
    X_test = X_test.astype('float32')
    X_test = torch.from_numpy(X_test)
    X_test = X_test.cuda()
    out = knn(X_test)
    del X_test
    return out



In [15]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

cvCount = 10
kForKnn = 5
numberOfKTry = 8
accuracyCount = np.zeros(numberOfKTry)
startK = 5
kUpgrade = 4
for kdTreePart in kdTreeArr:
    
    feature = kdTreePart[:, :-1]
    target = kdTreePart[:, -1]

    print("feature " + str(data_np.shape))

    print("target " + str(target.shape))


    #part 0
    X_trainAndVal, X_test, y_trainAndVal, y_test = train_test_split(feature, target, test_size = 0.2, random_state = 0)
    
    # Feature Scaling
    sc = StandardScaler()
    
    for cv in range(cvCount):
        X_train, X_val, y_train, y_val = train_test_split(X_trainAndVal, y_trainAndVal, test_size = 0.2, random_state = 0)
        print( "X_train "+str(X_train.shape))
        
        X_train = sc.fit_transform(X_train) 
        X_val = sc.transform(X_val)
        X_train = normalize_ecnodings(X_train)
        knn.weight = X_train.T
        batchSize = 5
        lenval=len(X_val) // batchSize + 1
        valindex=0
        for i in range(batchSize):
            xValBatch=X_val[valindex : valindex + lenval]
            yValBatch=y_val[valindex : valindex + lenval]
            out = similarity(xValBatch)
            
            index = 0
            out=out.cpu().detach().numpy()
            print("similarity shape "+str(out.shape))
            for kTry in range(startK, numberOfKTry * kUpgrade + startK, kUpgrade):
                accuracyCount[index] += select_k(out, kTry,X_train , yValBatch)
                index += 1
            valindex += lenval
            del out,X_train,X_val,y_train,y_val

feature (762104, 21)
target (190526,)
X_train (121936, 20)
similarity shape (6097, 121936)


IndexError: index 28903 is out of bounds for axis 0 with size 20

In [ ]:
accuracyCount /= (cvCount  * len(kdTreeArr) * batchSize)
bestKIndex = accuracyCount.argmax(axis=0) 
bestK = startK + bestKIndex * kUpgrade

In [ ]:
print(accuracyCount)